# --- `Day 8`: Seven Segment Search ---

In [145]:
import aocd
import re
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations, permutations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [97]:
def parse_line(line): 
    before, after = line.split(" | ")
    return (before.split(), after.split())
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [ ]:
final_input = parse_input(aocd.get_data(day=8, year=2021))
print(final_input[:5])

In [106]:
test_input = parse_input('''\
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce
''')

print(test_input)

[(['be', 'cfbegad', 'cbdgef', 'fgaecd', 'cgeb', 'fdcge', 'agebfd', 'fecdb', 'fabcd', 'edb'], ['fdgacbe', 'cefdb', 'cefbgd', 'gcbe']), (['edbfga', 'begcd', 'cbg', 'gc', 'gcadebf', 'fbgde', 'acbgfd', 'abcde', 'gfcbed', 'gfec'], ['fcgedb', 'cgb', 'dgebacf', 'gc']), (['fgaebd', 'cg', 'bdaec', 'gdafb', 'agbcfd', 'gdcbef', 'bgcad', 'gfac', 'gcb', 'cdgabef'], ['cg', 'cg', 'fdcagb', 'cbg']), (['fbegcd', 'cbd', 'adcefb', 'dageb', 'afcb', 'bc', 'aefdc', 'ecdab', 'fgdeca', 'fcdbega'], ['efabcd', 'cedba', 'gadfec', 'cb']), (['aecbfdg', 'fbg', 'gf', 'bafeg', 'dbefa', 'fcge', 'gcbea', 'fcaegb', 'dgceab', 'fcbdga'], ['gecf', 'egdcabf', 'bgf', 'bfgea']), (['fgeab', 'ca', 'afcebg', 'bdacfeg', 'cfaedg', 'gcfdb', 'baec', 'bfadeg', 'bafgc', 'acf'], ['gebdcfa', 'ecba', 'ca', 'fadegcb']), (['dbcfg', 'fgd', 'bdegcaf', 'fgec', 'aegbdf', 'ecdfab', 'fbedc', 'dacgb', 'gdcebf', 'gf'], ['cefg', 'dcbef', 'fcge', 'gbcadfe']), (['bdfegc', 'cbegaf', 'gecbf', 'dfcage', 'bdacg', 'ed', 'bedf', 'ced', 'adcbefg', 'gebcd'],

## Solution 1

In [115]:
def solve_1(input):
    total = 0
    for line in input:
        _, output = line
        
        lengths = [len(w) for w in output]
        total += count(l == 2 or l == 4 or l == 3 or l == 7
                       for l in lengths)
    return total

solve_1(test_input)

26

In [ ]:
f"Solution 1: {solve_1(final_input)}"

## Solution 2

In [130]:
def solve_2(input):
    total = 0
    for line in input:
        data, output = line
        
        numbers = []
        results = defaultdict(set)
        for i in data:
            signals = set(i)
            numbers.append(signals)
            l = len(signals)
            if l == 2:
                results[1] = signals
            elif l == 4:
                results[4] = signals
            elif l == 3:
                results[7] = signals
            elif l == 7:
                results[8] = signals
        
        # set 3
        for i in numbers:
            if len(i) == 5:
                z = i.intersection(results[1])
                if len(z) == 2:
                    results[3] = i
        
        # set 9
        for i in numbers:
            if len(i) == 6:
                z = i.intersection(results[3])
                if len(z) == 5:
                    results[9] = i
        # set 0
        for i in numbers:
            if len(i) == 6:
                if i != results[9]:
                    z = i.intersection(results[1])
                    if len(z) == 2:
                        results[0] = i
                    
        # set 6
        for i in numbers:
            if len(i) == 6:
                if i != results[0] and i != results[9]:
                    results[6] = i
                    
        # set 5
        for i in numbers:
            if len(i) == 5:
                if i != results[3]:
                    z = i.intersection(results[6])
                    if len(z) == 5:
                        results[5] = i
                        
        # set 2
        for i in numbers:
            if len(i) == 5:
                if i != results[3] and i != results[5]:
                    results[2] = i
        
        digits = ""
        for z in output:
            y = set(z)
            
            for x in range(10):
                if y == results[x]:
                    digits += str(x)
        #print(digits)
        total += int(digits)
    return total
    
solve_2(test_input)

61229

In [ ]:
f"Solution 2: {solve_2(final_input)}"

In [158]:
for aa in permutations("abc"):
    print(dict(zip("abc", aa)))

{'a': 'a', 'b': 'b', 'c': 'c'}
{'a': 'a', 'b': 'c', 'c': 'b'}
{'a': 'b', 'b': 'a', 'c': 'c'}
{'a': 'b', 'b': 'c', 'c': 'a'}
{'a': 'c', 'b': 'a', 'c': 'b'}
{'a': 'c', 'b': 'b', 'c': 'a'}


In [159]:
def decode(mapping, segments):
    mapped = ""
    for segment in segments:
        mapped += mapping[segment]
    return "".join(sorted(mapped)) 

def bruteForce(input):
    digits = {
        '0': "abcefg",
        '1': "cf",
        '2': "acdeg",
        '3': "acdfg",
        '4': "bcdf",
        '5': "abdfg",
        '6': "abdefg",
        '7': "acf",
        '8': "abcdefg",
        '9': "abcdfg"
    }
    
    count = 0
    for line in input:
        before, after = line
        
        for perm in permutations("abcdefgh"):
            wires = dict(zip("abcdefgh", perm))
            
            ok = True
            for segments in before:
                mapped = decode(wires, segments)
                
                if mapped not in digits.values():
                    ok = False
            if ok:
                result = ""
                for segments in after:
                    mapped = decode(wires, segments)
                    d = [k for k,v in digits.items() if v == mapped]
                    result += d[0]
                #print(result)
                count += int(result)
                break
    return count
        
bruteForce(test_input)

61229

In [ ]:
bruteForce(final_input)